# Likelihood Revisit

In [81]:
import pandas as pd

In [121]:
df = pd.read_pickle('test.pkl')

In [123]:
df[['TEMP', 'Lagged_Activity', 'Years_Schooling']].head(100)

TEMP  Lagged_Activity  Years_Schooling
Identifier Period                                        
0          0        1.0                1               12
           1        1.0                1               13
           2        1.0                1               14
1          0        0.0                0                8
           1        1.0                1                9
           2        3.0                3                9
2          0        0.0                0                8
           1        3.0                3                8
           2        3.0                3                8
3          0        1.0                1               12
           1        0.0                0               12
           2        3.0                3               12
4          0        0.0                0                8
           1        0.0                0                8
           2        0.0                0                8
5          0        1.0                1               12
           1        1.0                1               13
           2        1.0                1               14
6          0        0.0                0                8
           1        0.0                0                8
           2        3.0                3                8
7          0        0.0                0                8
           1        3.0                3                8
           2        1.0                1                9
8          0        0.0                0                8
           1        0.0                0                8
           2        3.0                3                8

In [120]:
is_lagged = df.loc[(slice(None), 0), 'Years_Schooling'] >= 10
is_lagged[is_lagged].index.get_level_values(0)

Int64Index([0, 3, 5], dtype='int64', name='Identifier')

In [117]:
index = list(is_lagged.index.get_level_values(0))
df.loc[(index, 0), 'TEMP']  = 1

In [118]:
df['TEMP']

Identifier  Period
0           0         1.0
            1         1.0
            2         1.0
1           0         1.0
            1         1.0
            2         3.0
2           0         1.0
            1         3.0
            2         3.0
3           0         1.0
            1         0.0
            2         3.0
4           0         1.0
            1         0.0
            2         0.0
5           0         1.0
            1         1.0
            2         1.0
6           0         1.0
            1         0.0
            2         3.0
7           0         1.0
            1         3.0
            2         1.0
8           0         1.0
            1         0.0
            2         3.0
Name: TEMP, dtype: float64